In [1]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import glob
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [3]:
train_data = pd.read_csv(r'train.csv')
train_data.shape

(10396, 37)

In [4]:
train_data["label"].value_counts()

0    5445
1    4951
Name: label, dtype: int64

In [3]:
train_data 

,eventId,startTime,transProtocol,appProtocol,name,srcAddress,srcPort,destAddress,destPort,srcGeoCity,...,requestUrl,httpReferer,accessAgent,responseCode,requestBody,requestHeader,responseHeader,requestContentType,responseContentType,label
0,292670958385078277,*** 15:16:44,TCP,http,HTTP请求访问,*.20.56.*,43052,*.215.158.*,80,NaN,...,"/hdip/static/isee.js/<img/src=""//xss.bxss.me/t...",NaN,MicroMessenger Client,400.0,NaN,"Accept:*/*,Cache-Control:no-cache,Connection:c...","Connection:close,Content-Type:application/octe...",application/octet-stream,application/octet-stream,0
1,292676887412228098,*** 09:58:07,TCP,http,HTTP请求访问,*.11.33.*,49769,*.50.2.*,80,NaN,...,/statistics,http://*.50.2.*/wiki/pages/viewpage.action?pag...,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,204.0,NaN,"Host:*.50.2.*,Connection:keep-alive,Accept:app...","Server:nginx/1.16.*,***,Content-Type:applicati...",NaN,application/json,1
2,291939933730643973,*** 09:03:59,TCP,http,HTTP请求访问,*.20.56.*,28117,*.71.3.*,80,NaN,...,/api/admin/public/gateInfo,http://admin.antivirus-ncov.dev.wochanye.com/,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,200.0,NaN,"Host:admin.antivirus-ncov.dev.wochanye.com,Use...","***,Content-Type:application/json;charset=UTF-...",application/json;charset=UTF-8,application/json;charset=UTF-8,0
3,292673842473820162,*** 17:08:59,TCP,http,HTTP请求访问,*.11.33.*,61812,*.200.143.*,80,NaN,...,/statistics,NaN,netdisk;6.8.9.1;PC;PC-Windows;10.0.18362;Windo...,200.0,\r\n----iphone_BOUNDARY--\r\nContent-Dispositi...,"Host:xacu02.baidupcs.com,Range:bytes=198574080...","***,Content-Type:application/octet-stream,Conn...",NaN,application/octet-stream,1
4,288571275298418693,*** 16:06:45,TCP,http,HTTP请求访问,*.20.14.*,54467,*.228.67.*,80,NaN,...,/jsonapi,http://mini.flash.2144.com/20190806/mini-main-...,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,200.0,...............c.>.f7..'.\r\nOy.....+f..?p.8.c...,"Accept:*/*,Referer:http://mini.flash.2144.com/...","Server:openresty,Connection:keep-alive,***,Con...",NaN,image/gif,1
5,292669920374018052,*** 14:34:19,TCP,http,HTTP请求访问,*.11.33.*,63782,*.50.2.*,80,NaN,...,/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr,http://*.50.2.*/wiki/pages/resumedraft.action?...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6...,200.0,"['[""[43,65,[33,\\""zsiiqEJSrBL79fgaqVE8laE\\"",\...","Host:*.50.2.*,Connection:keep-alive,Content-Le...","Server:nginx/1.16.*,***,Content-Type:applicati...",NaN,application/javascript;charset=UTF-8,1
6,291940780512333828,*** 12:36:34,TCP,http,HTTP请求访问,*.20.56.*,12267,*.111.160.*,80,NaN,...,/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr,NaN,NeteaseMusic 7.0.10/1514 (iPhone; iOS 13.3.1; ...,200.0,"{""stayTime"":7,""isFromHubei"":false,""isFromWenzh...","Host:clientlog3.music.163.com,Content-Type:mul...","Server:nginx,***,Content-Type:application/json...",multipart/form-data; charset=utf-8; boundary=0...,application/json;charset=UTF-8,0
7,293026483368210437,*** 11:08:33,TCP,http,HTTP请求访问,*.20.56.*,41634,*.20.120.*,18120,NaN,...,/res/v2,http://*.20.120.*:18120/,Mozilla/5.0 (Linux; Android 6.0.1; MuMu Build/...,404.0,"<?php%20echo%20""Anonymous_"".""woopra_test"";%20?>","Host:*.20.120.*:18120,Content-Length:814,autho...","Server:nginx,***,Content-Type:text/html,Conten...",NaN,text/html,0
8,293019145279410180,*** 18:01:04,TCP,http,HTTP请求访问,*.11.31.*,35840,*.163.26.*,80,NaN,...,/api/feedback/client/log,NaN,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,200.0,...............(.>..X..v.b.X...uTy*...!....oK$...,"Host:www.qq.com,User-Agent:Mozilla/4.0 (compat...","Content-Length:362,",application/octet-stream,NaN,1
9,292680843985104898,*** 12:28:24,TCP,http,HTTP请求访问,*.11.45.*,58227,*.192.137.*,80,NaN,...,/customimage/tile,NaN,Post_Multipart,200.0,NaN,"User-Agent:Post_Multipart,Host:*.192.137.*,Acc...","Server:nginx,***,Content-Type:application/octe...",multipart/form-data; boundary=----------------...,application/octet-stream,0


In [4]:
train_data.isnull().sum()
#每一列有几个缺失值

eventId                    0
startTime                  0
transProtocol              0
appProtocol                0
name                       0
srcAddress                 0
srcPort                    0
destAddress                0
destPort                   0
srcGeoCity             10396
srcGeoAddress          10396
srcGeoLatitude         10396
srcGeoLongitude        10396
destGeoCountry             6
destGeoRegion              6
destGeoCity             3260
destGeoAddress         10396
destGeoLatitude         3224
destGeoLongitude        3224
catOutcome               122
txId                       0
bytesOut                   0
bytesIn                    0
destHostName              12
requestMethod              0
httpVersion                9
requestUrlQuery            0
requestUrl                 0
httpReferer             5505
accessAgent              378
responseCode             122
requestBody             3722
requestHeader              8
responseHeader           140
requestContent

In [5]:
train_data["transProtocol"].value_counts()
train_data["appProtocol"].value_counts()
#train_data["name"].value_counts()

http    10396
Name: appProtocol, dtype: int64

In [6]:
train_data1 = train_data.drop(["srcGeoCity","srcGeoAddress","srcGeoLatitude","srcGeoLongitude","destGeoAddress","transProtocol","appProtocol","name"],axis=1)

In [7]:
train_data1

,eventId,startTime,srcAddress,srcPort,destAddress,destPort,destGeoCountry,destGeoRegion,destGeoCity,destGeoLatitude,...,requestUrl,httpReferer,accessAgent,responseCode,requestBody,requestHeader,responseHeader,requestContentType,responseContentType,label
0,292670958385078277,*** 15:16:44,*.20.56.*,43052,*.215.158.*,80,中国,广东,深圳,114.085947,...,"/hdip/static/isee.js/<img/src=""//xss.bxss.me/t...",NaN,MicroMessenger Client,400.0,NaN,"Accept:*/*,Cache-Control:no-cache,Connection:c...","Connection:close,Content-Type:application/octe...",application/octet-stream,application/octet-stream,0
1,292676887412228098,*** 09:58:07,*.11.33.*,49769,*.50.2.*,80,局域网,局域网,NaN,NaN,...,/statistics,http://*.50.2.*/wiki/pages/viewpage.action?pag...,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,204.0,NaN,"Host:*.50.2.*,Connection:keep-alive,Accept:app...","Server:nginx/1.16.*,***,Content-Type:applicati...",NaN,application/json,1
2,291939933730643973,*** 09:03:59,*.20.56.*,28117,*.71.3.*,80,中国,浙江,杭州,120.153576,...,/api/admin/public/gateInfo,http://admin.antivirus-ncov.dev.wochanye.com/,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,200.0,NaN,"Host:admin.antivirus-ncov.dev.wochanye.com,Use...","***,Content-Type:application/json;charset=UTF-...",application/json;charset=UTF-8,application/json;charset=UTF-8,0
3,292673842473820162,*** 17:08:59,*.11.33.*,61812,*.200.143.*,80,中国,陕西,西安,108.948024,...,/statistics,NaN,netdisk;6.8.9.1;PC;PC-Windows;10.0.18362;Windo...,200.0,\r\n----iphone_BOUNDARY--\r\nContent-Dispositi...,"Host:xacu02.baidupcs.com,Range:bytes=198574080...","***,Content-Type:application/octet-stream,Conn...",NaN,application/octet-stream,1
4,288571275298418693,*** 16:06:45,*.20.14.*,54467,*.228.67.*,80,中国,江苏,无锡,120.301663,...,/jsonapi,http://mini.flash.2144.com/20190806/mini-main-...,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,200.0,...............c.>.f7..'.\r\nOy.....+f..?p.8.c...,"Accept:*/*,Referer:http://mini.flash.2144.com/...","Server:openresty,Connection:keep-alive,***,Con...",NaN,image/gif,1
5,292669920374018052,*** 14:34:19,*.11.33.*,63782,*.50.2.*,80,局域网,局域网,NaN,NaN,...,/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr,http://*.50.2.*/wiki/pages/resumedraft.action?...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6...,200.0,"['[""[43,65,[33,\\""zsiiqEJSrBL79fgaqVE8laE\\"",\...","Host:*.50.2.*,Connection:keep-alive,Content-Le...","Server:nginx/1.16.*,***,Content-Type:applicati...",NaN,application/javascript;charset=UTF-8,1
6,291940780512333828,*** 12:36:34,*.20.56.*,12267,*.111.160.*,80,中国,浙江,杭州,120.153576,...,/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr,NaN,NeteaseMusic 7.0.10/1514 (iPhone; iOS 13.3.1; ...,200.0,"{""stayTime"":7,""isFromHubei"":false,""isFromWenzh...","Host:clientlog3.music.163.com,Content-Type:mul...","Server:nginx,***,Content-Type:application/json...",multipart/form-data; charset=utf-8; boundary=0...,application/json;charset=UTF-8,0
7,293026483368210437,*** 11:08:33,*.20.56.*,41634,*.20.120.*,18120,局域网,局域网,NaN,NaN,...,/res/v2,http://*.20.120.*:18120/,Mozilla/5.0 (Linux; Android 6.0.1; MuMu Build/...,404.0,"<?php%20echo%20""Anonymous_"".""woopra_test"";%20?>","Host:*.20.120.*:18120,Content-Length:814,autho...","Server:nginx,***,Content-Type:text/html,Conten...",NaN,text/html,0
8,293019145279410180,*** 18:01:04,*.11.31.*,35840,*.163.26.*,80,中国,上海,上海,121.472644,...,/api/feedback/client/log,NaN,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,200.0,...............(.>..X..v.b.X...uTy*...!....oK$...,"Host:www.qq.com,User-Agent:Mozilla/4.0 (compat...","Content-Length:362,",application/octet-stream,NaN,1
9,292680843985104898,*** 12:28:24,*.11.45.*,58227,*.192.137.*,80,中国,河南,郑州,113.665412,...,/customimage/tile,NaN,Post_Multipart,200.0,NaN,"User-Agent:Post_Multipart,Host:*.192.137.*,Acc...","Server:nginx,***,Content-Type:application/octe...",multipart/form-data; boundary=----------------...,application/octet-stream,0


In [8]:
train_data['transProtocol'].value_counts(normalize = True, dropna = False).head()

TCP    1.0
Name: transProtocol, dtype: float64

In [9]:
train_data['appProtocol'].value_counts(normalize = False, dropna = False).head()

http    10396
Name: appProtocol, dtype: int64

In [10]:
train_data['name'].value_counts(normalize = True, dropna = False).head()

HTTP请求访问    1.0
Name: name, dtype: float64

In [11]:
train_data['srcAddress'].value_counts(normalize = True, dropna = False).head()#种类不多

*.20.56.*    0.278761
*.11.44.*    0.221720
*.11.33.*    0.093786
*.11.45.*    0.054348
*.20.80.*    0.038669
Name: srcAddress, dtype: float64

In [12]:
train_data['srcPort'].value_counts(normalize = True, dropna = False).head() 

25463    0.001347
63380    0.001250
63551    0.001154
24635    0.001154
60615    0.001154
Name: srcPort, dtype: float64

In [13]:
train_data['destAddress'].value_counts(normalize = True, dropna = False).head() 

*.20.120.*     0.142843
*.50.2.*       0.086476
*.153.4.*      0.068103
*.71.3.*       0.060215
*.236.118.*    0.045594
Name: destAddress, dtype: float64

In [14]:
train_data['destPort'].value_counts(normalize = True, dropna = False).head() 

80       0.851193
18120    0.138996
39620    0.003270
7107     0.001828
8080     0.001250
Name: destPort, dtype: float64

In [15]:
train_data['destGeoCountry'].value_counts(normalize = True, dropna = False).head() 

中国     0.687668
局域网    0.309061
美国     0.001347
NaN    0.000577
爱尔兰    0.000481
Name: destGeoCountry, dtype: float64

In [16]:
train_data['destGeoRegion'].value_counts(normalize = True, dropna = False).head() 

局域网    0.309061
浙江     0.289053
北京     0.158619
上海     0.113890
江苏     0.031358
Name: destGeoRegion, dtype: float64

In [17]:
train_data['destGeoLatitude'].value_counts(normalize = True, dropna = False).head() 

NaN           0.310119
120.153576    0.260966
116.405285    0.158619
121.472644    0.113890
113.665412    0.023663
Name: destGeoLatitude, dtype: float64

In [18]:
train_data['destGeoLongitude'].value_counts(normalize = True, dropna = False).head() 

NaN          0.310119
30.287459    0.260966
39.904989    0.158619
31.231706    0.113890
34.757975    0.023663
Name: destGeoLongitude, dtype: float64

In [19]:
train_data['catOutcome'].value_counts(normalize = True, dropna = False).head() 

OK      0.673528
FALL    0.314736
NaN     0.011735
Name: catOutcome, dtype: float64

In [20]:
train_data['txId'].value_counts(normalize = True, dropna = False).head() 

0    0.660735
1    0.045210
2    0.024432
3    0.022028
4    0.017411
Name: txId, dtype: float64

In [21]:
train_data['bytesOut'].value_counts(normalize = True, dropna = False).head() 

714    0.151116
283    0.061851
388    0.026933
257    0.022797
397    0.022605
Name: bytesOut, dtype: float64

In [22]:
train_data['bytesIn'].value_counts(normalize = True, dropna = False).head() 

522     0.037995
469     0.023374
911     0.019046
725     0.016449
1134    0.013659
Name: bytesIn, dtype: float64

In [23]:
train_data['destHostName'].value_counts(normalize = True, dropna = False).head() 

*.20.120.*:18120                         0.138996
clientlog3.music.163.com                 0.090612
*.50.2.*                                 0.086476
admin.antivirus-ncov.dev.wochanye.com    0.060215
cxpt.hzwindow.com.cn                     0.045114
Name: destHostName, dtype: float64

In [24]:
train_data['requestMethod'].value_counts(normalize = True, dropna = False).head() 

POST       0.637168
GET        0.358215
HEAD       0.002212
PUT        0.002020
OPTIONS    0.000385
Name: requestMethod, dtype: float64

In [25]:
train_data['httpVersion'].value_counts(normalize = True, dropna = False).head() 

HTTP/1.1    0.997114
HTTP/1.0    0.002020
NaN         0.000866
Name: httpVersion, dtype: float64

In [26]:
train_data['requestUrlQuery'].value_counts(normalize = True, dropna = False).head() 

/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr      0.035783
/rest/sur                                             0.031454
/fsearch                                              0.026164
/statistics                                           0.012312
/wiki/rest/mywork/latest/status/notification/count    0.012216
Name: requestUrlQuery, dtype: float64

In [27]:
train_data['requestUrl'].value_counts(normalize = True, dropna = False).head() 

/api/auth/jwt/token                                 0.061658
/rest/sur                                           0.040015
/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr    0.039823
/api/admin/public/gateInfo                          0.038573
/fsearch                                            0.033090
Name: requestUrl, dtype: float64

In [27]:
train_data['httpReferer'].value_counts(normalize = True, dropna = False).head()

NaN                                                                                                                 0.529531
http://*.20.120.*:18120/                                                                                            0.138130
http://admin.antivirus-ncov.dev.wochanye.com/                                                                       0.034821
http://kaihua-h5-frontend.antivirus-ncov.dev.wochanye.com/activity/add                                              0.017411
http://*.50.2.*/wiki/pages/resumedraft.action?draftId=12750820&draftShareId=58022050-52aa-4b6b-a26a-23ac7523b161    0.015968
Name: httpReferer, dtype: float64

In [28]:
train_data['accessAgent'].value_counts(normalize = True, dropna = False).head()

Mozilla/5.0 (Linux; Android 6.0.1; MuMu Build/V417IR; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/52.0.2743.100 Mobile Safari/537.36 AliApp(DingTalk/4.5.15) com.alibaba.android.rimet/10573338 Channel/10005121 language/zh-CN    0.138996
NeteaseMusic 7.0.10/1514 (iPhone; iOS 13.3.1; zh_CN)                                                                                                                                                                                                 0.064255
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169                                                                                                                                          0.058388
NaN                                                                                                                                                                                                                                           

In [29]:
train_data['responseCode'].value_counts(normalize = True, dropna = False).head()
t=train_data['httpReferer']

200.0    0.641881
404.0    0.251539
500.0    0.039823
400.0    0.015294
204.0    0.014717
Name: responseCode, dtype: float64

In [30]:
train_data['requestBody'].value_counts(normalize = True, dropna = False).head()

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [31]:
train_data['requestHeader'].value_counts(normalize = True, dropna = False).head()

Host:*.50.2.*,Connection:keep-alive,Content-Length:0,User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.*,Accept:*/*,Origin:http://*.50.2.*,Referer:http://*.50.2.*/wiki/pages/resumedraft.action?draftId=12750820&draftShareId=58022050-52aa-4b6b-a26a-23ac7523b161,Accept-Encoding:gzip, deflate,Accept-Language:zh-CN,zh;q=0.*,Cookie:seraph.confluence=12615721%3Ad937ebd371378264a308e1d5216cd9299fa07218; confluence.browse.space.cookie=space-blogposts; JSESSIONID=0465B6A23D1D4026BB5E8F183CA35E26; mywork.tab.tasks=false; AJS.conglomerate.cookie="|hipchat.inapp.links.first.clicked.hoisan.bao=false"; crowd.token_key=5yjyz7-_GowWhY_LT2Ug_wAAAAAADAABaG9pc2FuLmJhbw,                                                                                                                                                                                                                                                                      

In [32]:
train_data['responseHeader'].value_counts(normalize = True, dropna = False).head()

Server:nginx,***,Content-Type:text/html,Content-Length:548,Connection:close,Strict-Transport-Security:max-age=31536000; includeSubDomains; preload,                                                                                                                                   0.096672
Server:nginx,***,Content-Type:application/json;charset=UTF-8,Transfer-Encoding:chunked,Connection:close,Vary:Accept-Encoding,Cache-Control:no-store, no-cache,Pragrma:no-cache,Expires:Thu, 01 Jan 1970 00:00:00 GMT,X-Application-Context:application:8181,Content-Encoding:gzip,    0.089842
Server:nginx,***,Content-Type:application/octet-stream,Transfer-Encoding:chunked,Connection:close,Cache-Control:no-cache,pragma:no-cache,                                                                                                                                             0.033282
Server:nginx/1.16.*,***,Content-Type:text/html,Content-Length:153,Connection:close,                                                        

In [33]:
train_data['requestContentType'].value_counts(normalize = True, dropna = False).head()

NaN                                  0.549154
application/x-www-form-urlencoded    0.052135
application/json;charset=UTF-8       0.050212
application/json                     0.045979
application/octet-stream             0.045787
Name: requestContentType, dtype: float64

In [34]:
train_data['responseContentType'].value_counts(normalize = True, dropna = False).head()


application/json;charset=UTF-8    0.220950
text/html                         0.166603
application/octet-stream          0.105425
NaN                               0.068488
text/plain; charset=utf-8         0.059061
Name: responseContentType, dtype: float64

In [28]:
train_data1.describe()

,eventId,srcPort,destPort,destGeoLatitude,destGeoLongitude,txId,bytesOut,bytesIn,responseCode,label
count,1.039600e+04,10396.000000,10396.000000,7172.000000,7172.000000,10396.000000,1.039600e+04,1.039600e+04,10274.000000,10396.000000
mean,2.917163e+17,46243.746345,2773.468449,118.124423,32.873783,9.713351,1.504173e+04,6.229609e+04,270.348647,0.476241
std,1.728837e+15,17950.697864,6647.386896,11.286796,4.527958,80.852490,2.345500e+05,5.237128e+06,102.572317,0.499459
min,2.885701e+17,1028.000000,80.000000,-122.329437,1.352083,0.000000,0.000000e+00,0.000000e+00,101.000000,0.000000
25%,2.919396e+17,40202.250000,80.000000,116.405285,30.287459,0.000000,2.590000e+02,4.790000e+02,200.000000,0.000000
50%,2.926673e+17,52954.000000,80.000000,120.153576,31.231706,0.000000,3.365000e+02,7.570000e+02,200.000000,0.000000
75%,2.926788e+17,59120.750000,80.000000,120.153576,36.094406,3.000000,7.140000e+02,1.379000e+03,404.000000,1.000000
max,2.937332e+17,65532.000000,52323.000000,139.731993,53.349805,4704.000000,1.048642e+07,5.311278e+08,503.000000,1.000000


In [29]:
one_value_fea = [col for col in train_data.columns if train_data[col].nunique() <= 1] 

In [30]:
numerical_fea = list(train_data1.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea,list(train_data1.columns)))
#filter(判断函数，序列)来滤除不符合判断函数的序列，返回容器，外面再加一个list（）转换为列表
#lambda匿名函数lambda [arg1 [,arg2,.....argn]]:expression

In [31]:


category_fea

['startTime',
 'srcAddress',
 'destAddress',
 'destGeoCountry',
 'destGeoRegion',
 'destGeoCity',
 'catOutcome',
 'destHostName',
 'requestMethod',
 'httpVersion',
 'requestUrlQuery',
 'requestUrl',
 'httpReferer',
 'accessAgent',
 'requestBody',
 'requestHeader',
 'responseHeader',
 'requestContentType',
 'responseContentType']

In [32]:
numerical_fea

['eventId',
 'srcPort',
 'destPort',
 'destGeoLatitude',
 'destGeoLongitude',
 'txId',
 'bytesOut',
 'bytesIn',
 'responseCode',
 'label']

In [33]:
category_fea.append('destPort')
category_fea.append('destGeoLatitude')
category_fea.append('destGeoLongitude')
category_fea.append('txId')
category_fea.append('responseCode')
category_fea.remove('startTime')
category_fea.remove('requestUrlQuery')
category_fea.remove('requestUrl')
category_fea.remove('requestBody')
category_fea.remove('requestHeader')
category_fea.remove('responseHeader')
category_fea.remove('requestContentType')

In [34]:
category_fea
#手动选出的特征

['srcAddress',
 'destAddress',
 'destGeoCountry',
 'destGeoRegion',
 'destGeoCity',
 'catOutcome',
 'destHostName',
 'requestMethod',
 'httpVersion',
 'httpReferer',
 'accessAgent',
 'responseContentType',
 'destPort',
 'destGeoLatitude',
 'destGeoLongitude',
 'txId',
 'responseCode']

In [35]:
for f in category_fea:
    print(f, '类型数：', train_data1[f].nunique())

srcAddress 类型数： 58
destAddress 类型数： 556
destGeoCountry 类型数： 8
destGeoRegion 类型数： 35
destGeoCity 类型数： 62
catOutcome 类型数： 2
destHostName 类型数： 568
requestMethod 类型数： 5
httpVersion 类型数： 2
httpReferer 类型数： 664
accessAgent 类型数： 607
responseContentType 类型数： 95
destPort 类型数： 14
destGeoLatitude 类型数： 66
destGeoLongitude 类型数： 66
txId 类型数： 148
responseCode 类型数： 19


In [36]:
for f in numerical_fea:
    print(f, '类型数：', train_data1[f].nunique())

eventId 类型数： 10396
srcPort 类型数： 7762
destPort 类型数： 14
destGeoLatitude 类型数： 66
destGeoLongitude 类型数： 66
txId 类型数： 148
bytesOut 类型数： 1044
bytesIn 类型数： 1751
responseCode 类型数： 19
label 类型数： 2


In [37]:
for data in [train_data1]:
    data = pd.get_dummies(data, columns=['srcAddress', 'destGeoCountry', 'destGeoRegion', 'destGeoCity','catOutcome',
                                         'requestMethod','httpVersion','responseContentType', 
                                         'destPort', 'destGeoLatitude', 'destGeoLongitude', 'txId', 'responseCode'], drop_first=True)
#进行哑变量编码，columns指定需要实现类别转换的列名，drop_first=Ture:丢弃第一列，把第一维的给丢了，主要用处是一些离散数据会有单独的一个维度来作为垃圾桶，例如 周一周二、、、、、周日和不确定，这个不确定当然就是我们不想要的

In [38]:
data

,eventId,startTime,srcPort,destAddress,bytesOut,bytesIn,destHostName,requestUrlQuery,requestUrl,httpReferer,...,responseCode_401.0,responseCode_403.0,responseCode_404.0,responseCode_405.0,responseCode_414.0,responseCode_422.0,responseCode_500.0,responseCode_501.0,responseCode_502.0,responseCode_503.0
0,292670958385078277,*** 15:16:44,43052,*.215.158.*,1269,252,szextshort.weixin.qq.com,"/hdip/static/isee.js/<img/src=""//xss.bxss.me/t...","/hdip/static/isee.js/<img/src=""//xss.bxss.me/t...",NaN,...,0,0,0,0,0,0,0,0,0,0
1,292676887412228098,*** 09:58:07,49769,*.50.2.*,129,709,*.50.2.*,/statistics?ver=1&vip=0&devuid=BDIMXV2-O_BFB25...,/statistics,http://*.50.2.*/wiki/pages/viewpage.action?pag...,...,0,0,0,0,0,0,0,0,0,0
2,291939933730643973,*** 09:03:59,28117,*.71.3.*,343,522,admin.antivirus-ncov.dev.wochanye.com,/api/admin/public/gateInfo?(select*from(select...,/api/admin/public/gateInfo,http://admin.antivirus-ncov.dev.wochanye.com/,...,0,0,0,0,0,0,0,0,0,0
3,292673842473820162,*** 17:08:59,61812,*.200.143.*,100,723,xacu02.baidupcs.com,/statistics,/statistics,NaN,...,0,0,0,0,0,0,0,0,0,0
4,288571275298418693,*** 16:06:45,54467,*.228.67.*,297,723,tongji.flash.cn,/jsonapi,/jsonapi,http://mini.flash.2144.com/20190806/mini-main-...,...,0,0,0,0,0,0,0,0,0,0
5,292669920374018052,*** 14:34:19,63782,*.50.2.*,293,725,*.50.2.*,/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr,/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr,http://*.50.2.*/wiki/pages/resumedraft.action?...,...,0,0,0,0,0,0,0,0,0,0
6,291940780512333828,*** 12:36:34,12267,*.111.160.*,333,1570,clientlog3.music.163.com,/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr,/wiki/synchrony-proxy/sockjs/v1/945/tqwtzbny/xhr,NaN,...,0,0,0,0,0,0,0,0,0,0
7,293026483368210437,*** 11:08:33,41634,*.20.120.*,1316,323,*.20.120.*:18120,/res/v2,/res/v2,http://*.20.120.*:18120/,...,0,0,1,0,0,0,0,0,0,0
8,293019145279410180,*** 18:01:04,35840,*.163.26.*,297,663,www.qq.com,/api/feedback/client/log,/api/feedback/client/log,NaN,...,0,0,0,0,0,0,0,0,0,0
9,292680843985104898,*** 12:28:24,58227,*.192.137.*,8175,264,*.192.137.*,/customimage/tile?&y=7&z=7&udt=20200225&scale=...,/customimage/tile,NaN,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df = data.iloc[:, 16:]
#取所有行，取第17列往后的所有列
train = np.array(df) 
#表转化为数列

In [40]:
df

,srcAddress_*.11.17.*,srcAddress_*.11.18.*,srcAddress_*.11.19.*,srcAddress_*.11.20.*,srcAddress_*.11.21.*,srcAddress_*.11.22.*,srcAddress_*.11.23.*,srcAddress_*.11.24.*,srcAddress_*.11.25.*,srcAddress_*.11.252.*,...,responseCode_401.0,responseCode_403.0,responseCode_404.0,responseCode_405.0,responseCode_414.0,responseCode_422.0,responseCode_500.0,responseCode_501.0,responseCode_502.0,responseCode_503.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
y = data.label
print(type(y))
x_train, x_test,y_train, y_test = train_test_split(train, y, test_size=0.3,random_state=0)#sklearn.model_selection.train_test_split（）
# (train_data,train_target,test_size=样本占比，如果是整数的话就是样本的数量, random_state=重复试验时0：每次随机数组不一样1：每次一样,stratify=y_train保持和train一样的类别分布比例) 
print(y_test)

<class 'pandas.core.series.Series'>
5482     0
5654     0
159      0
2359     1
9471     0
9454     1
7539     0
5176     0
5350     1
10361    0
9698     0
8550     1
9716     0
9768     1
9477     1
4597     1
9057     0
9829     1
8212     1
7730     0
2272     1
8906     1
4103     0
3474     1
2128     0
9821     1
8259     0
7222     0
7321     1
7328     1
        ..
8650     0
6978     1
7447     1
6092     1
8077     1
6451     1
9179     0
5447     0
8975     1
7151     1
8810     0
7367     1
1066     0
6967     0
5863     0
7041     1
6639     1
9207     0
6593     0
2025     0
7669     1
4674     0
2417     0
4936     0
6378     1
6749     1
2717     0
2177     0
6525     0
5865     1
Name: label, Length: 3119, dtype: int64


In [48]:
import xgboost as xgb
clf = xgb.XGBClassifier(
    objective= 'binary:logistic',
    eval_metric = 'auc',
#     scale_pos_weight = 5000/19723,
    learning_rate = 0.05,
    n_estimators = 600,
    gamma = 0.1,
    max_depth =6,
    reg_lambda = 1,
    reg_alpha=0.1, 
    subsample = 0.8,
    colsample_bytree = 0.7,
    min_child_weight =1,
    random_state =2020,
#     tree_method='hist'  # THE MAGICAL PARAMETER
)

clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)


In [ ]:
大数据队_ad_submission_1011=test1

In [248]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))
from sklearn.metrics import recall_score
print(recall_score(y_test, y_pred, average='macro'))
print(recall_score(y_test, y_pred, average='micro'))  
print(recall_score(y_test, y_pred, average='weighted'))  
print(recall_score(y_test, y_pred, average=None))  # [1. 0. 0.]

0.971465213209362
0.9709285718989433
0.971465213209362
0.971465213209362
[0.98818408 0.95367306]


In [182]:
print(train.shape)

(10396, 568)


In [123]:
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
for col in tqdm(['destAddress', 'destHostName','httpReferer','accessAgent']):
    le = LabelEncoder()
    le.fit(list(train_data1[col].astype(str).values))
    train_data1[col] = le.transform(list(train_data1[col].astype(str).values))
    print(train_data1[col])
print('Label Encoding 完成')


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 48.22it/s]

0        176
1         56
2        109
3        426
4        498
        ... 
10391    135
10392    163
10393    109
10394    321
10395     56
Name: destAddress, Length: 10396, dtype: int64
0        288
1        451
2         41
3        406
4        313
        ... 
10391    513
10392    560
10393     41
10394    113
10395    451
Name: destHostName, Length: 10396, dtype: int64
0        541
1        188
2        379
3        541
4        264
        ... 
10391    541
10392    541
10393    541
10394    501
10395     35
Name: httpReferer, Length: 10396, dtype: int64
0        561
1         83
2          9
3        395
4        210
        ... 
10391    253
10392    378
10393    211
10394    151
10395     83
Name: accessAgent, Length: 10396, dtype: int64
Label Encoding 完成


In [118]:
train_data1

,eventId,startTime,srcAddress,srcPort,destAddress,destPort,destGeoCountry,destGeoRegion,destGeoCity,destGeoLatitude,...,requestUrl,httpReferer,accessAgent,responseCode,requestBody,requestHeader,responseHeader,requestContentType,responseContentType,label
0,292670958385078277,*** 15:16:44,*.20.56.*,43052,290,80,中国,广东,深圳,114.085947,...,"/hdip/static/isee.js/<img/src=""//xss.bxss.me/t...",662,184,400.0,NaN,"Accept:*/*,Cache-Control:no-cache,Connection:c...","Connection:close,Content-Type:application/octe...",application/octet-stream,application/octet-stream,0
1,292676887412228098,*** 09:58:07,*.11.33.*,49769,436,80,局域网,局域网,NaN,NaN,...,/statistics,129,327,204.0,NaN,"Host:*.50.2.*,Connection:keep-alive,Accept:app...","Server:nginx/1.16.*,***,Content-Type:applicati...",NaN,application/json,1
2,291939933730643973,*** 09:03:59,*.20.56.*,28117,468,80,中国,浙江,杭州,120.153576,...,/api/admin/public/gateInfo,222,273,200.0,NaN,"Host:admin.antivirus-ncov.dev.wochanye.com,Use...","***,Content-Type:application/json;charset=UTF-...",application/json;charset=UTF-8,application/json;charset=UTF-8,0
3,292673842473820162,*** 17:08:59,*.11.33.*,61812,250,80,中国,陕西,西安,108.948024,...,/statistics,662,555,200.0,\r\n----iphone_BOUNDARY--\r\nContent-Dispositi...,"Host:xacu02.baidupcs.com,Range:bytes=198574080...","***,Content-Type:application/octet-stream,Conn...",NaN,application/octet-stream,1
4,288571275298418693,*** 16:06:45,*.20.14.*,54467,332,80,中国,江苏,无锡,120.301663,...,/jsonapi,459,420,200.0,...............c.>.f7..'.\r\nOy.....+f..?p.8.c...,"Accept:*/*,Referer:http://mini.flash.2144.com/...","Server:openresty,Connection:keep-alive,***,Con...",NaN,image/gif,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10391,291939810093121541,*** 08:56:35,*.20.56.*,24740,352,80,中国,浙江,杭州,120.153576,...,/api/auth/jwt/token,662,451,200.0,"['[{""name"":""jira.project.sidebar.issue.view.is...","Host:clientlog3.music.163.com,Content-Type:mul...","Server:nginx,***,Content-Type:application/json...",multipart/form-data; charset=utf-8; boundary=0...,application/json;charset=UTF-8,0
10392,293021960695255045,*** 03:13:44,*.20.4.*,55312,50,80,中国,北京,北京,116.405285,...,/wiki/rest/quickreload/latest/12750819,662,543,200.0,"['................P@...gx0...:....[.|.GG..?""xz...",Content-Type:application/x-www-form-urlencoded...,"Server:nginx/1.10.*,***,Content-Type:applicati...",application/x-www-form-urlencoded,application/json; charset=GBK,1
10393,291940715162054658,*** 12:33:36,*.20.56.*,5216,468,80,中国,浙江,杭州,120.153576,...,/secure/AjaxIssueEditAction!default.jspa,662,421,200.0,NaN,"Host:admin.antivirus-ncov.dev.wochanye.com,Acc...","Date:Tue, 18 Feb 2020 04:33:04 GMT,Age:0,X-Cac...",NaN,NaN,0
10394,291940646765641733,*** 12:29:03,*.20.56.*,3081,148,80,中国,浙江,杭州,120.153576,...,/sgim_netHot_20180731.zip,231,377,200.0,"{""stayTime"":7,""isFromHubei"":false,""isFromWenzh...","Host:cxpt.hzwindow.com.cn,User-Agent:Mozilla/5...","Server:nginx/1.16.*,***,Content-Type:text/html...",application/x-www-form-urlencoded,text/html;charset=utf-8,0


In [ ]:
ban